In [126]:
from project_helpers import load_csv_data, predict_labels
import numpy as np

In [116]:
#Compute sigmoid function
def sigmoid(z):
    return 1 / (1+np.exp(-z))

In [117]:
#Compute loss function
def loss_f(h, y):
    return (-y*np.log(h) - (1-y)*np.log(1-h)).mean()

In [118]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    
    w = initial_w
    h = 0
    
    for i in range(max_iters):
    
        #Compute x_t*w
        z = np.dot(tx, w)
        #Compute sigmoid of z
        h = sigmoid(z)

        #Compute gradient of loss function
        gradient = np.dot(tx.T, h-y) / y.shape[0]
        
        #Update w according to gradient
        w = w - gamma*gradient
    
    loss = loss_f(h, y)
    
    return (w, loss)

In [119]:
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    
    w = initial_w
    h = 0
    
    for i in range(max_iters):
    
        z = np.dot(tx, w)
        h = sigmoid(z)

        #The only difference with the previous function is the regularization constraint factored
        #in the gradient computation
        gradient = (np.dot(tx.T, h-y) / y.shape[0]) + 2*lambda_*w
        w = w - gamma*gradient
    
    loss = loss_f(h, y)
    
    return (w, loss)

In [120]:
data_path = 'Data/train.csv'
y, tx, ids = load_csv_data(data_path, sub_sample=True)

tx = np.concatenate((np.array([[1 for i in range(len(y))]]).T, tx), axis=1)

In [135]:
w, loss = logistic_regression(y, tx, np.ones(tx.shape[1]), 10000, 0.01)

/Users/Pepito/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Pepito/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Pepito/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Pepito/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


In [137]:
y_te, tx_te, ids_te = load_csv_data('Data/test.csv', sub_sample=True)

In [138]:
tx_te = np.concatenate((np.array([[1 for i in range(len(y_te))]]).T, tx_te), axis=1)
y_pred = predict_labels(w, tx_te)

In [143]:
np.count_nonzero(y_te == y_pred)

2272